In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import sys

In [2]:
# load data
df_train = pd.read_csv('train.csv')
train_y = df_train['Flood'].tolist()
train_x = df_train.iloc[:, 1:]

In [3]:
# train_x = train_x.to_numpy()

In [4]:
# load test
df_test = pd.read_csv('test.csv')
test_y = df_test['Flood'].tolist()
test_x = df_test.iloc[:, 1:]
# test_x = test_x.to_numpy()

In [5]:

N = 1500 # Number of persons in population
# D = len(train_x.columns) # Number of features in dataset
D = train_x.shape[1]
g = 10 # Number of generations
c = 0.6 # Self Introspection factor
r0 = 1
r1 = 0.4
r2 = 0.6
print(r1, r2)

0.4 0.6


In [6]:
fitness = np.zeros(N)
test_x.shape

(541, 11)

In [7]:
population = np.random.choice([0,1,2,3,4,5,6,7,8,9], (N,D), p=[0.16, 0.16, 0.16, 0.16, 0.16, 0.04, 0.04, 0.04, 0.04, 0.04])

In [8]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
global classifier
classifier = make_pipeline(StandardScaler(), SVC(gamma='auto'))
select = train_x.columns
selectno = len(train_x.columns)
classifier.fit(train_x.to_numpy(), train_y)
select_acc = classifier.score(test_x.to_numpy(), test_y) 
def fitness_function(pop):  #Fitness Function
  
  for i in range(N):
    new_train_x = train_x
    new_test_x = test_x
    
    global select
    global selectno
    global select_acc
    
    new_train_x = new_train_x.drop(train_x.columns[pop[i] < 4], axis = 1)
        
    new_test_x = new_test_x.drop(test_x.columns[pop[i] < 4], axis = 1)
        
    classifier.fit(new_train_x, train_y)  
    fitness[i] = classifier.score(new_test_x, test_y) 
    if (fitness[i] > select_acc):
      select = new_train_x.columns
      # print(select.shape)
      selectno = new_train_x.shape[1]
      select_acc = fitness[i]
    elif fitness[i] == select_acc and new_train_x.shape[1] < selectno:
      select = new_train_x.columns
      selectno = new_train_x.shape[1]
      
    print("\nPerson "+ str(i+1))
    print('column: ', new_train_x.columns)
    print("No. of Features Used = "+ str(new_train_x.shape[1])+ "/"+str(D)+"\nFitness = " + str(fitness[i]))
    print("Feature Used = ", end = " ") 
    
    #print(new_train_x.columns)

    print(new_train_x.shape[1])

In [9]:
select_acc

0.9870609981515711

In [10]:
gbest = 0
gbest_i = 0
def find_gbest():
  gbest = max(fitness)#This can be any function
  gbest_i = fitness.argmax()
  print("Best fitness value for the generation = "+str(gbest) + " Person " + str(gbest_i+1)+"\n")
find_gbest()

def cal_fitness(person):
  new_train_x = train_x
  
  new_test_x = test_x

  new_train_x = new_train_x.drop(train_x.columns[person < 4], axis = 1)
      
  new_test_x = new_test_x.drop(test_x.columns[person < 4], axis = 1)
      
  classifier.fit(new_train_x.to_numpy(), train_y)  
  return classifier.score(new_test_x.to_numpy(), test_y)

cal_fitness(population[0])

Best fitness value for the generation = 0.0 Person 1



0.977818853974122

In [11]:
def mutate():
  gworst_i = fitness.argmin()
  gworst = min(fitness)
  mut = np.random.randint(0,2,size=(1,D))[0]
  print("Mutating the Generation's Worst....Person "+ str(gworst_i+1))
  for i in range(D):
    if mut[i] > 0:
      mut[i] = population[gbest_i][i]
    else:
      mut[i] = population[gworst_i][i]
  if cal_fitness(mut) > gworst:
    population[gworst_i] = mut
    print("Person "+str(gworst_i)+" mutated")
  else:
    print("No Mutations in this generation")

In [12]:
## Improving Phase
# i = 1
def improve():
  print("Improving.......")
  for i in range(N):
    Xnew = population[i]
    print('Persona '+ str(i+1))
    for j in range(D):
      Xnew[j] = c * population[i][j] + r0 * (population[gbest_i][j] - population[i][j])
    try:
      if cal_fitness(Xnew) > fitness[i]:
        population[i] = Xnew
    except:
      print("Oops!", sys.exc_info()[0], "occurred.")
      print("Next entry.")

In [13]:

## Acquiring Phase
def acquire():
  random_person = np.random.randint(low=0, high=N)
  for i in range(N):
    if random_person == i:
      random_person = np.random.randint(low=0, high=N)
      i = i - 1
      continue
    X_new = population[i]
    if fitness[random_person] > fitness[i]:
      for j in range(D):
        X_new[j] = population[i][j] + r1*(population[random_person][j]-population[i][j]) + r2*(population[gbest_i][j]-population[i][j])
      if cal_fitness(X_new) > fitness[i]:
        population[i] = X_new
    else:
      for j in range(D):
        X_new[j] = population[i][j] + r1*(population[i][j]-population[random_person][j]) + r2*(population[gbest_i][j]-population[i][j])
      if cal_fitness(X_new) > fitness[i]:
        population[i] = X_new

In [14]:
try:
  for k in range(g):
    print("Generation "+ str(k+1) + "\n---------------")
    fitness_function(population)
    find_gbest()
    mutate()
    improve()
    acquire() 
except:
  print()
  print("........................")
  print("Optimal Solution Reached")
  print("........................")

Generation 1
---------------

Person 1
column:  Index(['CTI', 'Distance_river', 'Distance_road', 'Landuse2005',
       'Rainfall2005'],
      dtype='object')
No. of Features Used = 5/11
Fitness = 0.977818853974122
Feature Used =  5

Person 2
column:  Index(['CTI', 'Curvature', 'Distance_river', 'Distance_road', 'Landuse2005',
       'Slope', 'SPI'],
      dtype='object')
No. of Features Used = 7/11
Fitness = 0.9870609981515711
Feature Used =  7

Person 3
column:  Index(['CTI', 'Distance_river', 'Distance_road', 'Landuse2005'], dtype='object')
No. of Features Used = 4/11
Fitness = 0.9852125693160814
Feature Used =  4

Person 4
column:  Index(['SPI'], dtype='object')
No. of Features Used = 1/11
Fitness = 0.9186691312384473
Feature Used =  1

Person 5
column:  Index(['Curvature', 'Distance_river', 'SPI'], dtype='object')
No. of Features Used = 3/11
Fitness = 0.9630314232902033
Feature Used =  3

Person 6
column:  Index(['CTI', 'SPI'], dtype='object')
No. of Features Used = 2/11
Fitness = 